In [29]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
import pandas as pd
# import numpy as np

In [31]:
database_type = "/training"

#### Interpolating Data across patient timeseries

In [32]:
finalMerge = pd.read_csv('../eICU' + database_type + '/finalMerge.csv')
finalMerge = finalMerge.sort_values(by=['patientunitstayid', 'observationoffset'])
finalMerge

,patientunitstayid,observationoffset,temperature,heartrate,respiration,systemicsystolic,creatinine,wbcx1000,lactate,urineoutputbyweight,diagnosis
1648633,141764,-14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.003003,NaN
65,141764,4.0,NaN,105.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,141764,9.0,NaN,104.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
63,141764,14.0,NaN,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59,141764,19.0,NaN,98.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1634649,3353113,2671.0,NaN,87.0,27.0,NaN,NaN,NaN,NaN,NaN,NaN
1634633,3353113,2676.0,NaN,85.0,27.0,NaN,NaN,NaN,NaN,NaN,NaN
1634665,3353113,2681.0,NaN,84.0,26.0,NaN,NaN,NaN,NaN,NaN,NaN
1634621,3353113,2686.0,NaN,84.0,25.0,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
print("temperature: ", len(finalMerge['temperature'].unique()))
print("heartrate: ", len(finalMerge['heartrate'].unique()))
print("creatinine: ", len(finalMerge['creatinine'].unique()))
print("wbcx1000: ", len(finalMerge['wbcx1000'].unique()))
print("urineoutputbyweight: ", len(finalMerge['urineoutputbyweight'].unique()))
print("diagnosis: ", len(finalMerge['diagnosis'].unique()))

temperature:  666
heartrate:  198
creatinine:  945
wbcx1000:  1452
urineoutputbyweight:  10743
diagnosis:  3


In [34]:
idList = sorted(set(list(finalMerge['patientunitstayid'])))
cols = ['temperature', 'heartrate', 'respiration', 'systemicsystolic', 'creatinine', 'wbcx1000', 'lactate', 'urineoutputbyweight']
finalData = pd.DataFrame()

for id in idList:
    df = finalMerge[finalMerge['patientunitstayid'] == id]
    sepsisDiagnosisTime = df[df['diagnosis'] == 1].observationoffset
    countNull = df[cols].isnull().sum().sum()
    percentOfNull = (countNull / (len(df.index) * 8)) * 100
    # print(percentOfNull)

    if percentOfNull < 80:
        # Dropping patient rows after diganosis = 1 (True)
        if len(sepsisDiagnosisTime) != 0:
            sepsisDiagnosisTimeDict = sepsisDiagnosisTime.to_dict()
            sepsisDiagnosisTime = sepsisDiagnosisTimeDict[list(sepsisDiagnosisTimeDict.keys())[0]]
            df = df[df['observationoffset'] <= sepsisDiagnosisTime]
        # Interpolating Patient Data based on closet value
        df.loc[:, 'temperature'] = df['temperature'].interpolate(method='linear', limit_direction='both')
        df.loc[:, 'heartrate'] = df['heartrate'].interpolate(method='linear', limit_direction='both')
        df.loc[:, 'respiration'] = df['respiration'].interpolate(method='linear', limit_direction='both')
        df.loc[:, 'systemicsystolic'] = df['systemicsystolic'].interpolate(method='linear', limit_direction='both')
        df.loc[:, 'creatinine'] = df['creatinine'].interpolate(method='linear', limit_direction='both')
        df.loc[:, 'wbcx1000'] = df['wbcx1000'].interpolate(method='linear', limit_direction='both')
        df.loc[:, 'lactate'] = df['lactate'].interpolate(method='linear', limit_direction='both')
        df.loc[:, 'urineoutputbyweight'] = df['urineoutputbyweight'].interpolate(method='linear', limit_direction='both')
        finalData = finalData.append(df)

finalData

,patientunitstayid,observationoffset,temperature,heartrate,respiration,systemicsystolic,creatinine,wbcx1000,lactate,urineoutputbyweight,diagnosis
628,143870,7.0,NaN,44.0,86.0,111.0,0.89,11.7,NaN,3.870968,NaN
1677168,143870,10.0,NaN,43.0,80.5,112.5,0.89,11.7,NaN,3.870968,0.0
574,143870,12.0,NaN,42.0,75.0,114.0,0.89,11.7,NaN,3.870968,NaN
543,143870,17.0,NaN,41.0,78.0,113.0,0.89,11.7,NaN,3.870968,NaN
580,143870,22.0,NaN,41.0,73.0,113.0,0.89,11.7,NaN,3.870968,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1634649,3353113,2671.0,NaN,87.0,27.0,NaN,0.57,5.0,1.8,0.852273,NaN
1634633,3353113,2676.0,NaN,85.0,27.0,NaN,0.57,5.0,1.8,0.852273,NaN
1634665,3353113,2681.0,NaN,84.0,26.0,NaN,0.57,5.0,1.8,0.852273,NaN
1634621,3353113,2686.0,NaN,84.0,25.0,NaN,0.57,5.0,1.8,0.852273,NaN


In [35]:
finalData[cols].isnull().sum().sum()

3165015

#### Setting single missing cols to default human values

* temperature - 37
* heartrate - 80
* respiration - 20
* systemicsystolic - 120
* creatinine - 0.8
* wbcx1000 - 10
* lactate - 1.5
* urineoutputbyweight - 1.5

In [36]:
finalData['temperature'] = finalData['temperature'].fillna(finalData['temperature'].mean())
finalData['heartrate'] = finalData['heartrate'].fillna(finalData['heartrate'].mean())
finalData['respiration'] = finalData['respiration'].fillna(finalData['respiration'].mean())
finalData['systemicsystolic'] = finalData['systemicsystolic'].fillna(finalData['systemicsystolic'].mean())
finalData['creatinine'] = finalData['creatinine'].fillna(finalData['creatinine'].mean())
finalData['wbcx1000'] = finalData['wbcx1000'].fillna(finalData['wbcx1000'].mean())
finalData['lactate'] = finalData['lactate'].fillna(finalData['lactate'].mean())
finalData['urineoutputbyweight'] = finalData['urineoutputbyweight'].fillna(finalData['urineoutputbyweight'].mean())
finalData['diagnosis'] = finalData['diagnosis'].fillna(0)

In [37]:
finalData[cols].isnull().sum().sum()

0

In [38]:
finalData.sort_values(['patientunitstayid','observationoffset'])
finalData

,patientunitstayid,observationoffset,temperature,heartrate,respiration,systemicsystolic,creatinine,wbcx1000,lactate,urineoutputbyweight,diagnosis
628,143870,7.0,37.132325,44.0,86.0,111.000000,0.89,11.7,2.30671,3.870968,0.0
1677168,143870,10.0,37.132325,43.0,80.5,112.500000,0.89,11.7,2.30671,3.870968,0.0
574,143870,12.0,37.132325,42.0,75.0,114.000000,0.89,11.7,2.30671,3.870968,0.0
543,143870,17.0,37.132325,41.0,78.0,113.000000,0.89,11.7,2.30671,3.870968,0.0
580,143870,22.0,37.132325,41.0,73.0,113.000000,0.89,11.7,2.30671,3.870968,0.0
...,...,...,...,...,...,...,...,...,...,...,...
1634649,3353113,2671.0,37.132325,87.0,27.0,125.654566,0.57,5.0,1.80000,0.852273,0.0
1634633,3353113,2676.0,37.132325,85.0,27.0,125.654566,0.57,5.0,1.80000,0.852273,0.0
1634665,3353113,2681.0,37.132325,84.0,26.0,125.654566,0.57,5.0,1.80000,0.852273,0.0
1634621,3353113,2686.0,37.132325,84.0,25.0,125.654566,0.57,5.0,1.80000,0.852273,0.0


In [39]:
path = "../eICU/training/"
finalData.to_csv(path + "finalData.csv", sep=',', index=False, encoding='utf-8')
